In [1]:
!pip3 install minio
!pip3 install pandas
!sudo apt-get install -y mosquitto-clients

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mosquitto-clients is already the newest version (1.6.9-1).
0 upgraded, 0 newly installed, 0 to remove and 176 not upgraded.


In [2]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [3]:
#df_metrics = pd.DataFrame(columns=['Workload_size','Workload TPT','Average Workload TPT','Average FET','Average CT'])

In [4]:
scenario="WITHOUT_CONF"
user=3
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:9096/'


In [5]:
cmd = "./mqtt_client.sh "+str(user)
with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
    output1 = process.communicate()[0].decode("utf-8")
time.sleep(100)

In [6]:
df= pd.read_csv('mqtt.csv')
print(df)

   user                   intime
0     1  2022-08-31 11:46:25.890
1     2  2022-08-31 11:46:25.896
2     3  2022-08-31 11:46:25.908


In [7]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)
    client.remove_object(bucket_name, obj.object_name)

In [8]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime']).dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'])
df =df.sort_values(by="outtime")
display(df)

,user,intime,outtime
2,3,2022-08-31 11:46:25.908,2022-08-31 11:46:32.659
1,2,2022-08-31 11:46:25.896,2022-08-31 11:46:33.040
0,1,2022-08-31 11:46:25.890,2022-08-31 11:46:33.092


In [9]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))    


In [10]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep scaling-aeneas-mqtt | cut -c 93-97 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]


In [11]:
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
df['TPT']=(df['outtime']-df['intime']).dt.seconds
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
df['CT']= (df['TPT']).astype(float) - df['FET']
df =df.sort_values(by="outtime")
display(df)
df.to_csv(str(user)+"_"+scenario+".csv")

,user,intime,outtime,aeneas,tocloud,TPT,FET,CT
2,3,2022-08-31 11:46:25.908,2022-08-31 11:46:32.659,5.22,0.1574,6,5.3774,0.6226
1,2,2022-08-31 11:46:25.896,2022-08-31 11:46:33.040,5.19,0.0459,7,5.2359,1.7641
0,1,2022-08-31 11:46:25.890,2022-08-31 11:46:33.092,5.39,0.0451,7,5.4351,1.5649


In [12]:
metrics = {'workload_type':str(user)+"_"+scenario,'Workload_size':user,'Workload TPT':df['TPT'].sum(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean(),'Average CT':df['CT'].mean()}
print(metrics)

{'workload_type': '3_WITHOUT_CONF', 'Workload_size': 3, 'Workload TPT': 20, 'Average Workload TPT': 6.666666666666667, 'Average FET': 5.3494666666666655, 'Average CT': 1.3172000000000004}


In [13]:
pre_url = PROMETHEUS + '/api/v1/query?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time = (time.mktime(timestamp.timetuple()))
interval= int(df['TPT'].sum()) if int(df['TPT'].sum()) > 60  else 60


In [14]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [19]:
expr_cpu = '100 - (avg (irate(node_cpu_seconds_total{mode="idle"}['+str(interval)+'s])) * 100)&time='+str(time)
expr_memory = '1 - sum(node_memory_MemAvailable_bytes) / sum(node_memory_MemTotal_bytes)&time='+str(time)
expr_disk_read = 'sum(rate(node_disk_read_bytes_total['+str(interval)+'s]))&time='+str(time)
expr_disk_write =  'sum(rate(node_disk_written_bytes_total['+str(interval)+'s]))&time='+str(time)
expr_network_transmit= 'sum(rate(node_network_transmit_errs_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
expr_network_recieve='sum(rate(node_network_receive_packets_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
metrics['C_CPU'] = getdataprometheus(pre_url+expr_cpu)
metrics['C_RAM'] = getdataprometheus(pre_url+expr_memory)
metrics['C_disk_read']=getdataprometheus(pre_url+expr_disk_read)
metrics['C_disk_write']=getdataprometheus(pre_url+expr_disk_write)
metrics['C_network_recieve']=getdataprometheus(pre_url+expr_network_transmit)
metrics['C_network_transmit']=getdataprometheus(pre_url+expr_network_recieve)

SyntaxError: invalid syntax (2020921876.py, line 9)

In [16]:
df_metrics = pd.read_csv("metrics.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)


/tmp/ipykernel_4577/39894093.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


In [17]:
df_metrics.to_csv("metrics.csv")

In [18]:
display(df_metrics)

,Unnamed: 0,workload_type,workload_size,Workload_size,Workload TPT,Average Workload TPT,Average FET,Average CT,C_CPU,C_RAM,C_disk_read,C_disk_write,C_network_recieve,C_network_transmit
0,0.0,2_WITHOUT_CONF,NaN,2,11,5.500000,4.743900,0.7561,9.725,0.350635,4642.133333,311296,0,635.266667
1,NaN,3_WITHOUT_CONF,NaN,3,20,6.666667,5.349467,1.3172,8.955555555552976,0.3544229974447589,0,140083.2,0,546.3666666666667
